In [1]:
import pandas as pd
import numpy as np 
import tensorflow
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img 
from tensorflow.keras.models import Sequential 
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dropout, Flatten, Dense  
from tensorflow.keras import applications  
from tensorflow.keras.utils import to_categorical
from PIL import Image
from tqdm import tqdm
import time
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
vgg16 = applications.VGG16(include_top=False, weights='imagenet') 
format = [".jpg",".png",".jpeg",".JPG",".JPEG",".PNG"]
animals = ['bear','butterfly','cat','chicken', 'cow','dog','elephant', 'horse', 'rhinoceros','sheep','snake','spider', 'squirrel','tiger']

In [3]:
def build_model():
    num_classes=14
    model = Sequential()
    model.add(Flatten(input_shape=(7,7,512)))  
    model.add(Dense(100, activation=tensorflow.keras.layers.LeakyReLU(alpha=0.3)))  
    model.add(Dropout(0.5))  
    model.add(Dense(50, activation=tensorflow.keras.layers.LeakyReLU(alpha=0.3)))  
    model.add(Dropout(0.3)) 
    model.add(Dense(num_classes, activation='softmax'))  
    model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc']) 
    return model
    

In [4]:
def read_image(file_path):
    #print("[INFO] loading and preprocessing image...")  
    image = load_img(file_path, target_size=(224, 224))  
    image = img_to_array(image)  
    image = np.expand_dims(image, axis=0)
    image /= 255.  
    return image

In [5]:
def test_single_image(path,model):
    #animals = ['bear','butterfly','cat','chicken', 'cow','dog','elephant', 'horse', 'rhinoceros','sheep','snake','spider', 'squirell','tiger']
    images = read_image(path)
    #time.sleep(.5)
    bt_prediction = vgg16.predict(images)  
    preds = model.predict(bt_prediction) #model.predict_proba(bt_prediction)
    #for idx, animal, x in zip(range(0,14), animals , preds[0]):
    #    print("ID: {}, Label: {} {}%".format(idx, animal, round(x*100,2) ))
    class_predicted = np.argmax(preds,axis=1)
    perct = preds[0][class_predicted[0]]*100
    #print("\n"+path+"--->"+animals[class_predicted[0]]+", "+str(round(perct,2))+"%")
    return load_img(path),animals[class_predicted[0]],str(round(perct,2))+"%"

In [6]:
def check_files(tagpath,model):
    files = os.listdir(tagpath) #give the path of the directory
    file_list=[]
    label_list=[]
    confidence_list=[]
    #creating bootstrap samples
    random_files = np.random.choice(files, 200)
    for file in random_files:
        if file.endswith(tuple(format)):
            #print("\n"+tagpath+file)
            image,label,confidence=test_single_image(tagpath+file,model)
            file_list.append(file)
            label_list.append(label)
            confidence_list.append(confidence)
    d = {'file': file_list, 'label': label_list, 'confidence': confidence_list}
    df = pd.DataFrame(data=d)
    #print("\n[INFO]Tagging Completed")
    return df,len(file_list)
    

In [9]:
def compare_accuracy(tagpath,keyword_list,model_list):
    
    acc_list=[]
    animal_list=[]
    models=[]
    for animal in tqdm(keyword_list):
        print("Checking Accuracy for: "+animal)
        for model in model_list:
            print("Running model: "+model.split('.')[0])
            model_test=build_model()
            model_test.load_weights(model)
            #testing on the bootstrap samples
            for i in tqdm(range(500)):
                df1,total=check_files(tagpath+animal+"/",model_test)
                count1=df1[df1['label']==animal]
                #print("Testing Model: "+model)
                acc1=round(count1.count()[0]/total,4)
                #print("Accuracy: "+str(acc1*100))
                acc_list.append(acc1)
                animal_list.append(animal)
                models.append(model.split('.')[0])
        
    d = {'model': models, 'animal': animal_list,'accuracy': acc_list}
    df = pd.DataFrame(data=d)
    return df

In [ ]:
tagpath='/home/gupta.shes/animals/images/test/'
keyword_list = ['bear','sheep','dog','horse','elephant']#animals
model_list = ['top_model_weights_no_3d.h5','top_model_weights_with_3d.h5','top_model_weights_with_3d_2.h5']
df_final=compare_accuracy(tagpath,keyword_list,model_list)


  0%|          | 0/500 [00:00<?, ?it/s]

Checking Accuracy for: elephant
Running model: top_model_weights_no_3d



 97%|█████████▋| 484/500 [2:49:21<05:38, 21.19s/it]

In [ ]:
table = pd.pivot_table(df_final, values='accuracy', index=['animal'],columns=['model'], aggfunc=np.mean)

In [ ]:
ax = table.plot.bar(rot=0,figsize=(20,15))
table.plot(figsize=(12,10))

In [ ]:
table

In [ ]:
table.to_csv("table.csv")

In [ ]:
fig =ax.figure
fig.savefig("result.pdf")

In [ ]:
df_final.to_csv("acc_elephant_no_3d.csv")